Практическое домашнее задание к семинару от 02-02-2024:
1. Доделать НС из урока, добавить метод predict и проверить на своих собственных данных, оценить accuracy.

2. Как вариант, опционально кто сможет для решения этих входных данных обучите НС mlpclassifier от scikit-learn.

Сдача только через гитхаб либо ссылкой на блокнот Google Colab.


Часть 1: добавление метода predict() и проверка на своих собственных данных, оценка accuracy.

In [2]:
import numpy as np

In [3]:
def sigmoid(x):
  # Наша функция активации: f(x) = 1 / (1 + e^(-x))
  return 1 / (1 + np.exp(-x))

In [4]:
class Neuron:
  def __init__(self, weights, bias):
    self.weights = weights
    self.bias = bias

  def feedforward(self, inputs):
    # Умножаем входы на веса, прибавляем порог, затем используем функцию активации
    total = np.dot(self.weights, inputs) + self.bias
    return sigmoid(total)

In [5]:
weights = np.array([0, 1]) # w1 = 0, w2 = 1
bias = 4                   # b = 4
n = Neuron(weights, bias)

x = np.array([2, 3])       # x1 = 2, x2 = 3
print(n.feedforward(x))    # 0.9990889488055994

0.9990889488055994


In [24]:
class OurNeuralNetwork:
  def __init__(self):
    # Веса
    self.w1 = np.random.normal()
    self.w2 = np.random.normal()
    self.w3 = np.random.normal()
    self.w4 = np.random.normal()
    self.w5 = np.random.normal()
    self.w6 = np.random.normal()

    # Пороги
    self.b1 = np.random.normal()
    self.b2 = np.random.normal()
    self.b3 = np.random.normal()

  def feedforward(self, x):
    # x is a numpy array with 2 elements.
    h1 = sigmoid(self.w1 * x[0] + self.w2 * x[1] + self.b1)
    h2 = sigmoid(self.w3 * x[0] + self.w4 * x[1] + self.b2)
    o1 = sigmoid(self.w5 * h1 + self.w6 * h2 + self.b3)
    return o1

  ### функция для получения прогнозных значений
  def predict(self, data):
        predictions = []
        for x in data:
            predictions.append(round(self.feedforward(x)))
        return predictions

  def train(self, data, all_y_trues):
    '''
    - data - массив numpy (n x 2) numpy, n = к-во наблюдений в наборе.
    - all_y_trues - массив numpy с n элементами.
      Элементы all_y_trues соответствуют наблюдениям в data.
    '''
    learn_rate = 0.1
    epochs = 1000 # сколько раз пройти по всему набору данных

    for epoch in range(epochs):
      for x, y_true in zip(data, all_y_trues):
        # --- Прямой проход (эти значения нам понадобятся позже)
        sum_h1 = self.w1 * x[0] + self.w2 * x[1] + self.b1
        h1 = sigmoid(sum_h1)

        sum_h2 = self.w3 * x[0] + self.w4 * x[1] + self.b2
        h2 = sigmoid(sum_h2)

        sum_o1 = self.w5 * h1 + self.w6 * h2 + self.b3
        o1 = sigmoid(sum_o1)
        y_pred = o1

        # --- Считаем частные производные.
        # --- Имена: d_L_d_w1 = "частная производная L по w1"
        d_L_d_ypred = -2 * (y_true - y_pred)

        # Нейрон o1
        d_ypred_d_w5 = h1 * deriv_sigmoid(sum_o1)
        d_ypred_d_w6 = h2 * deriv_sigmoid(sum_o1)
        d_ypred_d_b3 = deriv_sigmoid(sum_o1)

        d_ypred_d_h1 = self.w5 * deriv_sigmoid(sum_o1)
        d_ypred_d_h2 = self.w6 * deriv_sigmoid(sum_o1)

        # Нейрон h1
        d_h1_d_w1 = x[0] * deriv_sigmoid(sum_h1)
        d_h1_d_w2 = x[1] * deriv_sigmoid(sum_h1)
        d_h1_d_b1 = deriv_sigmoid(sum_h1)

        # Нейрон h2
        d_h2_d_w3 = x[0] * deriv_sigmoid(sum_h2)
        d_h2_d_w4 = x[1] * deriv_sigmoid(sum_h2)
        d_h2_d_b2 = deriv_sigmoid(sum_h2)

        # --- Обновляем веса и пороги
        # Нейрон h1
        self.w1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w1
        self.w2 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w2
        self.b1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_b1

        # Нейрон h2
        self.w3 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w3
        self.w4 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w4
        self.b2 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_b2

        # Нейрон o1
        self.w5 -= learn_rate * d_L_d_ypred * d_ypred_d_w5
        self.w6 -= learn_rate * d_L_d_ypred * d_ypred_d_w6
        self.b3 -= learn_rate * d_L_d_ypred * d_ypred_d_b3

      # --- Считаем полные потери в конце каждой эпохи
      if epoch % 100 == 0:
        y_preds = np.apply_along_axis(self.feedforward, 1, data)
        loss = mse_loss(all_y_trues, y_preds)
        print("Epoch %d loss: %.3f" % (epoch, loss))

In [7]:
def mse_loss(y_true, y_pred):
  # y_true и y_pred - массивы numpy одинаковой длины.
  return ((y_true - y_pred) ** 2).mean()

In [8]:
def deriv_sigmoid(x):
  # Производная сигмоиды: f'(x) = f(x) * (1 - f(x))
  fx = sigmoid(x)
  return fx * (1 - fx)

In [53]:
def calculate_accuracy(true_values, predicted_values):
    """
    Calculate accuracy metric.

    Аргументы функции:
    true_values (array-like): массив истинных значений.
    predicted_values (array-like): массив прогнозных значений.

    Возращаемые значения:
    float: метрика точности (accuracy).
    """
    # проверка соответствие аргументов
    if len(true_values) != len(predicted_values):
        raise ValueError("Длины массива истинных значений и массива прогнозных значений должны быть равными.")

    # Непосредственно расчет accuracy
    correct_predictions = sum(1 for true, pred in zip(true_values, predicted_values) if true == pred)
    #print("Сумма правильно предсказанных значений: ", {correct_predictions})
    total_predictions = len(true_values)
    accuracy = correct_predictions / total_predictions

    return accuracy

In [9]:
### функция для нормализации и масштабирования веса и роста человека

def normalize_height_weight(data):
    # Приведение входных данных к numpy массиву
    data = np.array(data)

    # Проверка размерности входных данных
    if data.ndim != 2 or data.shape[1] != 2:
        raise ValueError("Аргументом функции должен быть двумерный массив NumPy (матрица), где первый столбец - рост, а второй - вес.")

    ### нормализация данных
    normalized_data = (data - np.mean(data, axis=0)) / np.std(data, axis=0)

    ### масштабирование данных в значения из диапазона [0, 1]
    ### подходит для применения в наборах данных с большим количеством входных переменных - в данной задаче я это не применял
    #scaled_data = (normalized_data - np.min(normalized_data, axis=0)) / (np.max(normalized_data, axis=0) - np.min(normalized_data, axis=0))

    return normalized_data


In [58]:
# Определим набор данных
data = np.array([
  [54.5, 165.1],  # Алиса
  [65.44, 183],   # Боб
  [62.2, 178],   # Чарли
  [49, 152], # Диана
])
all_y_trues = np.array([
  1, # Алиса
  0, # Боб
  0, # Чарли
  1, # Диана
])

In [29]:
### нормализуем входные данные
scaled_input_data = normalize_height_weight(data)
print("Подготовленные входные данные:")
print(scaled_input_data)

Подготовленные входные данные:
[[-0.50984076 -0.3674422 ]
 [ 1.18807641  1.11893415]
 [ 0.68521977  0.70374523]
 [-1.36345543 -1.45523718]]


In [30]:
network = OurNeuralNetwork()
network.train(scaled_input_data, all_y_trues)

Epoch 0 loss: 0.240
Epoch 100 loss: 0.053
Epoch 200 loss: 0.017
Epoch 300 loss: 0.009
Epoch 400 loss: 0.006
Epoch 500 loss: 0.004
Epoch 600 loss: 0.003
Epoch 700 loss: 0.003
Epoch 800 loss: 0.002
Epoch 900 loss: 0.002


In [46]:
# Ввод собственных данных
my_data = np.array([
  [91, 181],  # Андрей
  [50, 160],   # Лиза
  [75, 180],   # Даниил
  [55, 172], # Александра
])
my_y_trues = np.array([
  0, # Андрей
  1, # Лиза
  0, # Даниил
  1, # Александра
])

In [47]:
### нормализуем собственные входные данные
scaled_input_my_data = normalize_height_weight(my_data)
print("Подготовленные собственные входные данные:")
print(scaled_input_my_data)

Подготовленные собственные входные данные:
[[ 1.42104854  0.9217866 ]
 [-1.08488652 -1.57595774]
 [ 0.44312266  0.8028464 ]
 [-0.77928468 -0.14867526]]


In [50]:
### Строим прогноз на основе ранее обученной модели для собственных данных c округлением, чтобы получить
predictions = network.predict(scaled_input_my_data)
print("Predictions:", predictions)

Predictions: [0, 1, 0, 1]


In [54]:
accuracy_value = calculate_accuracy(my_y_trues, predictions)
print("Accuracy:", accuracy_value)

Accuracy: 1.0


Часть 2: для решения этих входных данных обучение нейронной сети с помощью модуля MLPClassifier из библиотеки scikit-learn.

In [69]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [70]:
#### нормализация входных данных средствами библиотеки scikit-learn
scaler = StandardScaler()
X_scaled = scaler.fit_transform(data)
print(X_scaled)

[[-0.50984076 -0.3674422 ]
 [ 1.18807641  1.11893415]
 [ 0.68521977  0.70374523]
 [-1.36345543 -1.45523718]]


In [90]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, all_y_trues, test_size = 0.2, random_state = 42)
#print(f'{X_train=}')
#print(f'{X_test=}')
#print(f'{y_train=}')
#print(f'{y_test=}')

In [91]:
#### Строим модель многослойного перцептрона из двух скрытых слоёв в 100 и 50 нейронов соответственно
MLP_model = MLPClassifier(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', random_state = 42)
MLP_model.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(100, 50), random_state=42)

In [92]:
y_pred = MLP_model.predict(X_test)
print("Predictions:", y_pred)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Predictions: [0]
Accuracy: 1.0


In [93]:
### Проверка работы модели на собственных данных
my_scaler = StandardScaler()
my_X_scaled = my_scaler.fit_transform(my_data)
print(my_X_scaled)

my_new_predictions = MLP_model.predict(my_X_scaled)
print("Predictions:", my_new_predictions)

my_accuracy = accuracy_score(my_y_trues, my_new_predictions)
print("Accuracy:", my_accuracy)

[[ 1.42104854  0.9217866 ]
 [-1.08488652 -1.57595774]
 [ 0.44312266  0.8028464 ]
 [-0.77928468 -0.14867526]]
Predictions: [0 1 0 1]
Accuracy: 1.0
